In [1]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [2]:
import numpy as np
import pandas as pd
import sys

10/26/2023 07:07:44 PM-INFO: NumExpr defaulting to 8 threads.


In [3]:
sys.path.append("../")
sys.path.append("../../WP3_Benchmark/tnbs/BTC_04_PH/PH")

In [4]:
from compare import do_mps, do_myqlm

/home/gferro/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'qat.qlmaas'

In [5]:
nqubits = 12
depth = 2

In [6]:
pdf_mps, angles_mps = do_mps(nqubits, depth)
pdf_myqlm, angles_myqlm = do_myqlm(nqubits, depth)

NameError: name 'do_mps' is not defined

In [ ]:
indices = np.isclose(
    pdf_mps["PauliCoefficients"].astype(float), 
    pdf_myqlm["PauliCoefficients"].astype(float)
)

In [ ]:
pdf_myqlm[~indices]["PauliCoefficients"]

In [ ]:
pdf_mps[~indices]["PauliCoefficients"]

In [ ]:
(pdf_myqlm[~indices]["PauliCoefficients"] - pdf_mps[~indices]["PauliCoefficients"]).sum()

In [9]:
16 * 16

256

In [ ]:
from ansatz_mps import ansatz, get_angles
from parent_hamiltonian_mps import PH_MPS
ph_conf = {"save": False}
ph_ob_mps = PH_MPS(mps, True, **ph_conf)

In [ ]:
# MPS uisng My code
depth = 3
nqubits = 4 
mps = ansatz(nqubits, depth, get_angles(depth))
ph_conf = {"save": False}
ph_ob_mps = PH_MPS(mps, True, **ph_conf)
ph_ob_mps.local_ph()

In [ ]:
ph_ob_mps.pauli_pdf

In [ ]:
def do_mps(nqubits, depth):
    # MPS uisng My code
    mps = ansatz(nqubits, depth, get_angles(depth))
    ph_conf = {"save": False}
    ph_ob_mps = PH_MPS(mps, True, **ph_conf)
    ph_ob_mps.local_ph()
    return ph_ob_mps.pauli_pdf

In [ ]:
do_mps(nqubits, depth)

In [ ]:
sys.path.append("../")
import tensornetworks as tn
import tn_quantum_circuits as tnqc
import gates as gt
from tensornetworks import contract_indices, contract_indices_one_tensor, density_matrix_mps_contracion

In [ ]:
def apply_2qubit_gates(qubits, gates):
    """
    Executes product of tensor with a gate
    -o-o-o-o-o-..o-o-
     |   |   |     |
    """
    new_qubits = [0 for i in qubits]
    left = qubits[0]
    for i in range(1, len(qubits)):
        right = qubits[i]
        gate = gates[i-1]
        #new_qubits[i-1], left = phase_change(left, right, gate)
        new_qubits[i-1], left = tnqc.apply_2qubit_gate(left, right, gate)

    new_qubits[-1], new_qubits[0] = tnqc.apply_2qubit_gate(
        left, new_qubits[0], gates[-1])
     #new_qubits[-1], new_qubits[0] = phase_change(left, new_qubits[0], gates[-1])
    return new_qubits

def get_angles(depth):
    theta = np.pi/4.0
    delta_theta = theta / (depth + 1)
    angles = []
    for i in range(depth):
        angles.append([(2 * i + 1) * delta_theta, (2 * i + 2) * delta_theta])
    return angles     

def ansatz(nqubits, depth, angles):
    # Intitial State
    zeroket = np.zeros((1, 2, 1))
    zeroket[0][0][0] = 1
    zeroket = zeroket.astype(complex)
    #Initial State
    mps_ = [zeroket] * nqubits
    for depth_ in range(depth):
        # First Layer
        gates = [gt.x_rotation(angles[depth_][0]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
        ent_gates = [gt.controlz() for i in mps_]
        mps_ = apply_2qubit_gates(mps_, ent_gates)
        gates = [gt.z_rotation(angles[depth_][1]) for i in mps_]
        mps_ = tnqc.apply_local_gate(mps_, gates)
    return mps_

In [ ]:
# MPS uisng My code
depth = 3
nqubits = 4 
mps = ansatz(nqubits, depth, get_angles(depth))
pdf_zalo = tnqc.get_state_from_mps(mps)

In [ ]:
from parent_hamiltonian_mps import PH_MPS

In [ ]:
ph_conf = {"save": False}
ph_ob_mps = PH_MPS(mps, True, **ph_conf)

In [ ]:
ph_ob_mps.local_ph()

In [ ]:
ph_ob_mps.pauli_pdf

In [ ]:


def reduced_rho_mps(mps, free_indices, contraction_indices):
    i = 0
    tensor_out = mps[i]
    # Starting Tensor for Denisty Matrix
    
    if i in free_indices:
        tensor_out = contract_indices(tensor_out, tensor_out.conj(), [], [])
    elif i in contraction_indices:
        tensor_out = contract_indices(tensor_out, tensor_out.conj(), [1], [1])
    else:
        raise ValueError("Problem with site i: {}".format(i))
    
    for i in range(1, len(mps)):
        tensor = mps[i]
        if i in free_indices:
            tensor = contract_indices(tensor, tensor.conj(), [], [])
        elif i in contraction_indices:
            tensor = contract_indices(tensor, tensor.conj(), [1], [1])
        else:
            raise ValueError("Problem with site i: {}".format(i))        
        
        tensor_out = density_matrix_mps_contracion(tensor_out, tensor)
        
    tensor_out= contract_indices_one_tensor(tensor_out, [(0, 2), (3, 5)])
    
    return tensor_out   

def get_local_reduced_matrix(state, qb_pos):
    """
    Given a MPS representation of a input state and position qubit
    (qb_pos) computes the minimum local reduced density matrix from
    qb_pos qubit such that its null space can be computable:
    i.e. rank(local_rho) < dim(local_rho)

    Parameters
    ----------

    state : numpy array
        MPS representation of an state
    qb_pos : int
        position of the qubit for computing the reduced density matrix

    Returns
    _______

    local_qubits : list
        list with the qubits affected by the reduced density matrix
    local_rho : numpy array
        array with the reduced density matrix for the input qbit
        position
    """
    logger.debug("Computing local reduced density matrix: qb_pos: %d", qb_pos)
    nqubit = len(state)
    state_index = list(range(nqubit))
    if qb_pos not in state_index:
        text = "qb_pos: {} NOT IN: {}".format(qb_pos, state_index)
        raise ValueError(text)
    group_qbits = 1
    stop = True
    while stop:
        # Iteration for grouping one qubit more in each step of the loop
        free_indices = [(qb_pos + k)%nqubit for k in range(group_qbits + 1)]
        logger.debug("\t free_indices: %s", free_indices)
        # The contraction indices are built
        contraction_indices = [
            i for i in state_index if i not in free_indices
        ]
        logger.debug("\t contraction_indices: %s", contraction_indices)
        # Computing the reduced density matrix
        rho = reduced_rho_mps(state, free_indices, contraction_indices)
        # Computes the rank of the obtained reduced matrix
        rank = np.linalg.matrix_rank(rho)
        logger.debug("\t rank: %d. Dimension: %d", rank, len(rho))
        if rank < len(rho):
            # Now we can compute a null space for reduced density operator
            logger.debug("\t Grouped Qubits: %s", free_indices)
            # Store the local qubits for each qubit
            local_qubits = free_indices
            # Store the local reduced density matrices for each qubit
            local_rho = rho
            stop = False
        group_qbits = group_qbits + 1

        if group_qbits == nqubit:
            stop = False
            local_rho = rho
            local_qubits = free_indices
        logger.debug("\t STOP: %s", stop)
    return local_qubits, local_rho


In [ ]:
get_local_reduced_matrix(mps, 1)

### MyQLM

In [ ]:
sys.path.append("../../WP3_Benchmark/tnbs/BTC_04_PH/PH")

In [ ]:
from ansatzes import run_ansatz
from parent_hamiltonian import PH

In [ ]:
conf = {
    "nqubits": nqubits,
    "depth": depth,
    "ansatz" : "simple01",
    "qpu_ansatz": "c",
    "t_inv": True,
    "folder":  "/EASQC/",
    "save": False,
    "solve" : True,
    "submit": False
}
solved_ansatz = run_ansatz(**conf)
pdf = solved_ansatz["state"]
ph_conf = {"save": False}
ph_ob = PH(pdf[["Amplitude"]], True, **ph_conf)
ph_ob.local_ph()
ph_ob.pauli_pdf

In [ ]:
np.isclose(
    ph_ob_mps.pauli_pdf["PauliCoefficients"].astype(float),
    ph_ob.pauli_pdf["PauliCoefficients"].astype(float)
).all()

In [ ]:
ph_ob.pauli_pdf["PauliCoefficients"].astype(float)


In [ ]:
ph_ob_mps.pauli_pdf["PauliCoefficients"]

In [ ]:
np.isclose?